In [1]:
import pandas as pd
import numpy as np

In [2]:
log = pd.read_csv('log.csv', header=None)
users = pd.read_csv('users.csv',sep='\t',encoding='KOI8-R')

In [3]:
# Теперь нужно сделать так, чтобы с колонками было удобно работать.
log.columns = ['user_id','time','bet','win']
users.columns = ['user_id','email','geo']

In [4]:
log['time'] = log['time'].str.replace('[','')

In [ ]:
#  Посчитайте количество пропусков в столбце time.
log['time'].isna().sum()

<span style="color: orange; font-weight: bold; font-size:16pt">Удаление пропусков</span>

Пропуски можно удалять автоматически. Во многих случаях это правильно, так как данные с большим количеством пропусков часто не имеют смысла и не приносят никакой пользы. 

Удалять данные с пропусками можно с помощью метода ***dropna().***

Параметр axis в методе ***dropna()*** говорит методу, по какой оси удалять значения.  

Если нужно удалить строки, в которых встречается пропуск (NaN), следует указать axis=0.  Зачем это делать? Например, у нас из 1000 примеров данных про пользователей пропуски есть в пяти. Разумно их удалить, так как их количество пренебрежимо мало.

Если нужно удалить столбцы, в которых встречается пропуск (NaN), нужно указывать axis=1. Зачем? Иногда в одном конкретном столбце пропусков настолько много, что с ними просто не хочется возиться - смысла в них все равно почти нет. 

Еще один интересный параметр - ***subset***. Что он делает? Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре axis другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск. То же самое работает и наоборот: нужно поменять axis на 1 и вместо названий столбцов передавать индексы строк.

In [ ]:
# Удалите все столбцы, где есть пропуски. Запишите в поле, сколько осталось столбцов в данных после этого.
log1 = log.copy()
log1.dropna(axis=1)

In [ ]:
# Удалите все строки, где есть пропуски. Запишите в поле, сколько осталось строк в данных после этого.

log2 = log.copy()
log2.dropna(axis=0)

С данными в столбцах bet и win мы разберемся позже: пропуски в этих столбцах требуют особого подхода.

А сейчас:

если есть пропуски в столбце user_id - удалите столбец user_id,

если есть пропуски в столбце time - удалите столбец time.

Запишите в поле ответа, количество оставшихся столбцов в данных, после этих действий.

In [ ]:
empty_data = log.isna().sum()[log.isna().sum() > 0].reset_index()
empty_data.columns = ['Column', 'Count']
display(empty_data)

In [ ]:
log2 = log.copy()
log2 = log2.drop(['time'], axis=1)
log2

<span style="color: orange; font-weight: bold; font-size:16pt">Дубли</span>

Дубли - это повторяющиеся строки в данных. В сложных случаях строки могут быть практически одинаковые, но не совсем.

Самая частая причина очень банальна: дубли появляются из-за человеческих ошибок или невнимательности.

Например, при добавлении записи в систему вы случайно два раза нажали на кнопку "добавить". Если система позволяет иметь одинаковые записи, поздравляю - у вас в данных появились дубли.

Еще одна причина - слияние баз данных. Например, вы переносите телефоны из контактной книжки (физической, в которую заносили номера ручкой на бумагу, такие были популярны в прошлом) в телефон. Назвали в одном месте Сашу Сашей, а в другом Александром. Да, это тоже дубль, просто сам случай немного более сложный.

Сложным случаям типа последнего можно посвятить очень много времени, так что это материал для другого, более продвинутого курса по анализу данных. Скажу лишь, что в каждом конкретном случае подозрительные случаи нужно рассматривать отдельно, это требует кропотливой работы. 

Мы же рассмотрим простой случай, когда у вас в данных есть идентичные строки.

В pandas есть метод для удаления дублей (дубликатов) - **drop_duplicates()**. Он просто удаляет повторяющиеся строки.

У данного метода тоже есть параметр subset, в этом случае нужно передавать список содержащий названия столбцов.  

In [ ]:
# Удалите дубли среди столбцов user_id и time. Запишите в поле ниже, сколько осталось строк после удаления дублей.
log1 = log[['user_id', 'time']].copy()
log1 = log1.drop_duplicates()
log1

<span style="color: orange; font-weight: bold; font-size:16pt">Преобразование к datetime</span>

Для преобразования столбца с датой в виде текста в дату формата datetime (см. Модуль "С1. Работа с датами"), необходимо использовать метод to_datetime() в библиотеке pandas.

Внутрь метода нужно передать тот столбец, который требуется преобразовать.

In [ ]:
#  Уберите лишний символ, преобразуйте признак time к datetime. После этого найдите наибольшую дату и выведите ее без времени.

log5 = log.copy()
log5 = log5.dropna(axis = 0)
log5['time'] = pd.to_datetime(log5['time'])
log5['time'].max()

In [ ]:
logg = pd.read_csv("log.csv")  
logg = logg.dropna()  
logg.columns = ['user_id', 'time', 'bet', 'win']  
logg['time'] = logg['time'].apply(lambda x: x[1:])  
logg['time'] = pd.to_datetime(logg['time'])  
logg['time'] = logg.time.apply(lambda x: x.minute)
logg['time'].head() 

<span style="color: orange; font-weight: bold; font-size:16pt">Извлечение признаков времени</span>


**Извлечение признаков времени**

Для начала вспомним, что мы можем делать с datetime. Вот примеры атрибутов, по которым мы можем обращаться к данным объектам:

year: возвращает год
month: возвращает месяц
day: возвращает день
hour, minute, second - час, минута, секунда
dayofweek - день недели, от 0 до 6, где 0 - понедельник, 6 - воскресенье

Кроме них, есть и другие интересные атрибуты, советуем посмотреть здесь.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html

Ранее в курсе вы разбирали метод apply(). Он позволяет применить определенную функцию к каждому элементу в столбце.

В метод apply() можно передавать обычные и lambda-функции.

Например, если мы хотим получить столбец, в котором каждым значением будет год из другого столбца (это и есть feature engineering - создание новых признаков из старых), мы можем сделать следующее:

In [ ]:
log1 = log.copy()
log1['time'] = pd.to_datetime(log1['time'])  
year_column = log1['time'].apply(lambda x: x.year) 

Библиотека pandas позволяет использовать аксессор dt для упрощения подобной работы:

In [ ]:
year_colums = log1['time'].dt.year

Аксессор - это атрибут столбца, который хранит переменные типа Timestamp, то есть переменные, которые были строковым представлением времени, а затем изменены с помощью pd.to_datetime(). Если вы попытаетесь обратиться к dt у столбца, в котором лежит что-то отличное от времени, вы получите ошибку.
Чуть больше можно увидеть здесь (не забудьте посмотреть в исходный код).
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html


Вы можете пользоваться любым из предложенных выше способов. 


In [ ]:
# Используйте оригинальные данные log.csv, столбец time.
# Найдите минуту, которая встречалась в данных чаще всего. Введите ответ в поле ниже.
log2 = log.copy()
log2['time'] = pd.to_datetime(log2['time']) 

minute = log2['time'].dt.minute
minute.value_counts()

In [ ]:
# Найдите месяц, который встречался в данных реже всего. Введите ответ в поле ниже.
month = log2['time'].dt.month
month.value_counts(ascending = True)


In [ ]:
# Посчитайте, сколько дней в данных являются выходными (то есть субботой или воскресеньем). Введите ответ в поле ниже.
#V1
weekdays = log2['time'].dt.weekday
a = weekdays[weekdays == 5].value_counts()
b = weekdays[weekdays == 6].value_counts()
print(a)
print(b)
152+131

In [ ]:
# V2
log2['weekday'] = log2['time'].dt.weekday
#weekday_counts = log2.groupby('weekday').aggregate(sum)
log2[(log2['weekday'] == 5) | (log2['weekday'] == 6)].count()

In [ ]:
#V3
log2 = log.copy()
log2['time'] = pd.to_datetime(log['time']) # convert to date
log2['time'].dt.weekday.apply(lambda x: x==5 or x==6).sum()

In [ ]:
# Посчитайте, какое время дня встречается в данных реже всего. Введите ответ в поле ниже: ночь, утро, день или вечер.
# Договоримся, что с 0 до 5 часов - ночь, с 6 до 11 - утро, с 12 до 17 - день, с 18 до 23 - вечер.
# Подсказка: можно использовать value_counts(). Кроме этого, потребуется написать функцию, которая преобразует дату во время дня
log_d = log.copy()
#log_d = log_d.dropna(axis = 0)
log_d['time'] = pd.to_datetime(log_d['time'])
res = log_d['time'].dt.hour.value_counts().sort_values().reset_index()
# res сколько раз каждый час встречается в дата сете
print('Night', res.query('0 < index <= 5')['time'].sum())  # night
print('Morning', res.query('6 < index <= 11')['time'].sum())  # Morn
print('Day', res.query('12 < index <= 17')['time'].sum())  # Day
print('Evening', res.query('18 < index <= 23')['time'].sum())  # Even
display(res)

Давайте повторим то, что мы прошли в этой секции. Напишите код, который создаст признак hour из признака time в датасете log.csv. Для этого:

1. загрузите датасет log.csv в переменную log, дальше работать будем с ней;

2. установите имена столбцов: ['user_id', 'time', 'bet', 'win'];

3. избавьтесь от пропусков в log;

4. приведите переменную time к подходящему для извлечения признаков виду;

5. получите значение часа для каждой строки в переменной time и запишите в столбец hour в log.

Результатом будет таблица log со столбцом hour внутри.

In [ ]:
log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id','time','bet','win']
log = log.dropna(axis=0)
log['time'] = log['time'].str.replace('[','')
log['time'] = pd.to_datetime(log['time'])
log['hour'] = log.time.dt.hour


<span style="color: orange; font-weight: bold; font-size:16pt">Обработка пропусков</span>

Ранее мы говорили о пропусках - местах в данных, где по какой-то причине ничего нет.

Что мы делали с пропусками? Удаляли их.

На самом деле, удаление пропусков - довольно грубое решение, потому что мы можем случайно выбросить что-то полезное. Например, у вас есть данные о людях: пол, возраст, цвет глаз, город. При этом для большинства людей не записан цвет глаз. Удалять строки с пропусками может быть неудачным решением -  у нас почти не останется данных. Удалить столбец с цветом глаз - решение получше. Возможно, есть решение еще лучше - например, заполнить все пропуски цветом глаз "карие". Ответ на вопрос "Правильно ли будет так сделать?" можно получить после уточнения информации: в каких-то странах преобладает один цвет глаз, в каких-то - другой.

Поэтому довольно часто можно заполнить пропуски, сделав некоторые обоснованные предположения.

Сейчас мы познакомимся с самым простым способом заполнять пропуски - заполнением константой.

каждый пропуск в столбце мы заполним одним и тем же числом.

Почему мы вообще говорим о заполнении пропусков вместо того, чтобы просто удалить их? Потому что после удаления пропусков у нас останется слишком мало данных, а нам хочется получить какие-то инсайты из них.

<span style="color: orange; font-weight: bold; font-size:16pt">Заполнение константой</span>

Что такое константа? Это просто число.

Посмотрим на признак bet в наших данных (log.csv).

"Bet" означает "ставка".  Некоторые значения в данных заполнены цифрами - поэтому мы можем сделать предположение, что это сумма в рублях, т.е. ставка. 

Можно также предположить, что если в данных в этом месте пропуск - значит, человек не делал ставку. Другими словами, ставка в данном случае равна 0.

Чтобы заполнить пропуски в столбце каким-то значением, можно использовать метод *fillna()* у самого столбца. Аргументом этого метода будет число, которое появится на месте пропусков.

In [ ]:
# Посчитайте, сколько раз люди приходили, но не делали ставок. Для этого заполните пропуски в столбце 
# bet значением 0 и посчитайте количество таких значений.

log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id','time','bet','win']
log1 = log.copy()
log1['bet'] = log1['bet'].fillna(0)
log1['bet'].value_counts()

<span style="color: orange; font-weight: bold; font-size:16pt">Заполнение с помощью функции</span>

Теперь поработаем с признаком win, в котором тоже есть пропуски.

Иногда нужно заполнять пропуски не одним и тем же числом, а разными, в зависимости от какого-то условия. Перед нами именно этот случай.

Предположим, что если в признаке win находится пропуск, то выигрыша не было. Здесь два возможных случая:

Человек не делал ставки и ничего не выиграл. То есть просто пришел, посмотрел и ушел.
Человек делал ставку, но не выиграл. Значит, выигрыш на самом деле является отрицательным значением - это проигрыш.
Предлагаем вам написать метод, который заполнит пропуски в признаке win в соответствии с предположением выше. 

Для этого можно применить метод apply() ко всей таблице и передать ему функцию, которая вычисляет размер выигрыша (или проигрыша) по следующей схеме:

Если значение в столбце win существует (не пропуск) - вернуть это же значение. Это значит, что человек выиграл.
Если вместо значения в столбце win пропуск, вернуть 0.
На выходе получится столбец без пропусков. Следующим шагом будет замена старого столбца win на новый.

In [ ]:
def fillna_win(row):  
    if row.isna().win == True:
        return 0
    return row.loc['win'] 
    
new_win = log1.apply(lambda row: fillna_win(row), axis=1)  

# Заменяем старый столбец с пропусками на новый без пропусков  
log1['win'] = new_win 
log1

In [ ]:
# Посчитайте, сколько раз участники ставок проиграли деньги. То есть посчитайте количество строк, 
# для которых в столбце win находится отрицательное значение.
def fillna_win(row):  
    if row.win == 0:
        return 0 - row.loc['bet']
    return row.loc['win'] 
    
new_win = log1.apply(lambda row: fillna_win(row), axis=1)  

# Заменяем старый столбец с пропусками на новый без пропусков  
log1['win'] = new_win 
log1['win'].apply(lambda x: x<0).sum()


Используйте модифицированный в прошлой секции датасет log.csv. Результат запишите числом в поле ниже.

Подсказка: можно использовать sum().
Создайте признак net, хранящий сумму выигрыша с учетом ставки. Для этого используйте следующий алгоритм:
- если значение признака win меньше 0 - присвоить значение признака win признаку net;

- во всех остальных случаях - из значения признака win вычтите значение признака bet и полученное значение присвойте признаку net.

После этого посчитайте, у скольких людей выигрыш положительный.

In [ ]:
def net_net(row):
    if row.win < 0:
        return row.loc['win']
    return row.loc['win'] - row.loc['bet']
log1['net'] = log1.apply(lambda row: net_net(row),axis=1)
log1['net'].apply(lambda x: x>0).sum()

Используйте датасет log.csv, получившийся в результате выполнения предыдущего задания. Посчитайте среднее значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0. Результат округлите до целого, отбросив дробную часть.

Подсказка: можно использовать mean().

In [ ]:
log1[log1['net']>0]['net'].mean()

Посчитайте медианное значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0. Результат округлите до целого, отбросив дробную часть.

In [ ]:
log1[log1['net']>0]['net'].median()

Как можно посчитать среднее значение для столбца bet, не учитывая при подсчете пропуски?

In [ ]:
log.bet.mean()
#log.bet.sum() / log.bet.dropna().shape[0]
#log['bet'].dropna().mean()
#np.mean(log.bet)

Используйте модифицированный исходный датасет log.csv.

При модификации датасета log.csv, пропущенные значения в столбцах bet и win замените на 0, cоздайте столбец net, хранящий сумму выигрыша с учетом ставки (для этого из столбца win поэлементно вычтите столбец bet и запишите в новый столбец).

Посчитайте, какой процент посещений букмекерской конторы оборачивался ставкой. Для этого поделите количество ставок (значений больше 0) на общее количество посещений конторы. Результат округлите до одного знака после запятой.

In [ ]:
round(len(log1[log1['bet']>0]['win'])/len(log1),3)*100

Используйте датасет log.csv, получившийся в результате модификации при выполнении первого задания этого блока.

Посчитайте среднее значение ставки (из столбца bet) в тех случаях, когда ставка была сделана. Запишите результат, отбросив дробную часть.
Подсказка: можно использовать mean().

In [ ]:
log1[log1['bet']>0]['bet'].mean()


Используйте датасет log.csv, получившийся в результате модификации при выполнении первого задания этого блока.

Посчитайте средний выигрыш (из столбца net) в тех случаях, когда ставка была сделана. В ответ запишите полученное число с отброшенной дробной частью.
Пояснение: выигрыш в данном случае означает изменение количества денег и может быть отрицательным. В таком случае это проигрыш.

Подсказка: можно использовать mean().

In [ ]:
log1[(log1['bet']>0)]['net'].mean()

Посчитайте среднее значение потерь при проигрыше (из столбца net). Результат округлите до целого, отбросив дробную часть

In [ ]:
log1[(log1['bet']>0)&(log1['net']<0)]['net'].mean()

Посчитайте, какой процент ставок заканчивается выигрышем, а какой - проигрышем. Сравните эти значения и ответьте, какое из них больше.

In [ ]:
print(len(log1[(log1['bet']>0)&(log1['net']<0)])/len(log1)*100) # Lose
print(len(log1[(log1['bet']>0)&(log1['net']>0)])/len(log1)*100) # win

Давайте повторим то, что мы прошли в этой секции. Напишите код, который узнает, чему была равна минимальная ставка и сколько людей сделали такую ставку. Для этого:

1. загрузите датасет log.csv;

2. посчитайте, чему равна минимальная ставка;

3. посчитайте, сколько раз была сделана минимальная ставка, и запишите результат в переменную min_bet_amount в виде целого числа.

In [5]:
log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id','time','bet','win']
log['bet'].min()
min_bet_amount = log[log['bet'] == log['bet'].min()]['bet'].count()
min_bet_amount

48

<span style="color: orange; font-weight: bold; font-size:16pt">Повторение merge/groupby</span>

Повторим часть предобработки, которую мы должны были выполнить ранее:


In [6]:
log['bet'] = log['bet'].fillna(0)
def fillna_win(row):  
    if row.isna().win == True:
        return 0
    return row.loc['win'] 
    
new_win = log.apply(lambda row: fillna_win(row), axis=1)  

# Заменяем старый столбец с пропусками на новый без пропусков  
log['win'] = new_win 
log

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,0.0,0.0
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,0.0,0.0
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,0.0,0.0
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,0.0,0.0
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,0.0,0.0
...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,0.0
996,#error,NaN,10054.0,29265.0
997,#error,NaN,10454.0,0.0
998,#error,NaN,1000.0,0.0


In [7]:
def fillna_win(row):  
    if row.win == 0:
        return 0 - row.loc['bet']
    return row.loc['win'] 
    
new_win = log.apply(lambda row: fillna_win(row), axis=1)  

# Заменяем старый столбец с пропусками на новый без пропусков  
log['win'] = new_win 
log['win'].apply(lambda x: x<0).sum()

347

In [8]:
def net_net(row):
    if row.win < 0:
        return row.loc['win']
    return row.loc['win'] - row.loc['bet']
log['net'] = log.apply(lambda row: net_net(row),axis=1)
log['net'].apply(lambda x: x>0).sum()

138

In [9]:
users.user_id = users.user_id.apply(lambda x: x.lower()) 
log['time'] = log['time'].str.replace('[','')
log = log[log.user_id != '#error']    
log['user_id'] = log['user_id'].str.split(' - ').apply(lambda x: x[1])

<ipython-input-9-972241e40c73>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['user_id'] = log['user_id'].str.split(' - ').apply(lambda x: x[1])


Tеперь объединим данные с помощью метода pd.merge():

In [4]:
df = pd.merge(log, users, on='user_id')
df

KeyError: 'user_id'

**Groupby**

In [11]:
df.groupby('user_id').win.median().median()

0.0

В данном случае мы группируем данные по признаку user_id.

После этого мы в каждой группе выбираем признак win.

Затем мы берем медиану каждой группы по признаку win и на выходе получаем таблицу, в которой индексом является признак user_id. В этой таблице единственный столбец - медиана по каждой группе (то есть по каждому пользователю).

Наконец, последний вызов median() дает нам медиану по предыдущему столбцу, то есть возвращает одно число.

Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа (отбросьте дробную часть).

Посчитайте медиану баланса по каждому пользователю. Для этого сгруппируйте по пользователям, возьмите признак net, просуммируйте по каждому пользователю и получите медиану.

In [ ]:
df.groupby('user_id').net.sum().median()

Сколько раз в среднем каждый человек приходит, не делая ставок, при условии, что у этого человека все-таки есть хотя бы одна ставка? Например: Человек посетил букмекерскую контору 5 раз из них 1 раз сделал ставку, 4 раза нет - условие выполняется. Человек посетил букмекерскую контору 5 раз из них ни разу ставку не сделал - условие не выполняется. Для того, чтобы узнать это, просуммируйте в каждой группе количество записей со ставкой, равной 0, и поделите на общее количество групп. Если при этом в группе нет записей со ставкой больше 0, считаем количество записей в данной группе равным 0.

In [ ]:
zero=df[df['bet'] == 0]
mid = zero.groupby('user_id')['bet'].count().reset_index()
mid.mean()
#mid.sort_values('bet').mean

Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа - количества дней.

Сколько в среднем времени проходит между появлением человека в сервисе и первой ставкой? Считать нужно только тех, кто делал ставку. Для того, чтобы узнать это, напишите метод, считающий минимальное время среди ставок, равных 0, и минимальное время среди ставок больше 0. После этого верните разницу между вторым и первым числом. Пройдитесь по всем группам в цикле. Если в группе нет ставок больше 0, пропустите эту группу. Просуммируйте разницу во времени для каждой группы (с помощью метода, описанного выше) и поделите на количество групп, которые вы не пропустили.


In [3]:
df['time'] = pd.to_datetime(df['time'])
who_bet = df[df['bet']>0]
lis = [] # create a list for those who bet
for i in who_bet['user_id']:
    if i in lis:
        continue
    else:
        lis.append(i)
        
who_d_bet = df[df['bet']==0]


a = who_bet.groupby('user_id')['time'].min()
b = who_d_bet.groupby('user_id')['time'].min()
#for i in a['user_id']:
    #if i in lis:
        #print(i)
(a-b).mean()


    

NameError: name 'df' is not defined

Используйте датасет, который получился в результате всех преобразований в прошлой секции (в том числе, заполнение пропусков).

Ответ запишите в поле ниже в виде одного слова, с большой буквы.

Наибольший суммарный выигрыш среди всех городов имеет Москва. Посчитайте следующий за ней город. Для этого сгруппируйте по городам, возьмите признак win, просуммируйте по каждому городу, отсортируйте и получите второй город.

In [ ]:
df.groupby('geo')['win'].sum().sort_values(ascending = False)

Используйте датасет, который получился в результате всех преобразований в прошлой секции (в том числе, заполнение пропусков).

Подсказки:

1. Можно использовать методы min() и max().

2. Учитывайте, что минимальная ставка, это ставка, которая была сделана, т.е. если ставка равна нулю - значит ставки не было.

3. Ответ запишите в поле ниже в виде целого числа (нужно отбросить дробную часть).

Во сколько раз различаются максимальное и минимальное значение средней ставки по городам? Для того, чтобы это посчитать, нужно сгруппировать по городам, взять среднее от признака bet, найти максимальное и минимальное значения, затем поделить одно на другое.

In [ ]:
aux = df[df['bet']>0]
min_b = aux.groupby('geo')['bet'].mean().min()
max_b = aux.groupby('geo')['bet'].mean().max()
max_b/min_b

Давайте повторим все, что мы прошли в этой секции. Напишите код, который посчитает, сколько раз пользователи приходили в букмекерскую контору в каждом городе. Для этого:

1. загрузите датасеты log.csv и users.csv;

2. удалите user_id с ошибкой (#error) и приведите признак user_id к одному виду в обоих датасетах;

3. слейте два датасета в один по признаку user_id;

4. сгруппируйте данные по правильному признаку (какому - вам нужно понять самим), затем выберите user_id и воспользуйтесь функцией count() для подсчета наблюдений в каждой группе;

5. результат запишите в sample2 (объект Series).

In [ ]:
log = pd.read_csv('log.csv', header=None)
users = pd.read_csv('users.csv',sep='\t',encoding='KOI8-R')
log.columns = ['user_id','time','bet','win']
users.columns = ['user_id','email','geo']

users.user_id = users.user_id.apply(lambda x: x.lower()) 
log = log[log.user_id != '#error']    
log['user_id'] = log['user_id'].str.split(' - ').apply(lambda x: x[1])

df = pd.merge(log, users, on='user_id')

Trash

In [17]:
sample2 = df.groupby('geo')['user_id'].count()
sample2

geo
Арзангелтск         96
Воронеж             88
Екатеринбург        49
Ижевск              61
Казань              66
Краснодар           86
Красноярск          56
Москва              61
Пермь               55
Санкт-Петербург    115
Ставрополь          36
Тюмень              32
Хабаровск           60
Ярославль           89
Name: user_id, dtype: int64

In [16]:
df_bet = df.groupby('user_id')[['bet']].sum()
df_bet0 = df_bet[df_bet.bet == 0]
df_bet0

,bet
user_id,
